# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_gini(df, label):
    count = df[label].value_counts() #값들의 개수 구하기
    length = len(df[label])
    gini = 1
    for i in count:
        gini -= (i/length)**2 #pi^2
    
    return gini

In [11]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [24]:
def get_binary_split(df, attribute):
    result = []
    classes = df[attribute].unique().tolist()
    for i in range(len(classes)):
        #반복문을 순회 -> 복사본을 계속해서 만들어주면서 pop()을 이용해 해당 값과 아닌
        #값들의 list를 append
        copy_classes = classes.copy()
        deter = copy_classes.pop(i)
        result.append([deter])
        result.append(copy_classes)
    return result

In [25]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged', 'senior'],
 ['middle_aged'],
 ['youth', 'senior'],
 ['senior'],
 ['youth', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [41]:
def get_attribute_gini_index(df, attribute, label):
    length = len(df)
    binary_split = get_binary_split(df, attribute)
    result = {}
    for i in binary_split:
        temp_df1 = df[df[attribute].map(lambda x: x in i)] #해당 class를 갖고 있는 df
        temp_df2 = df[df[attribute].map(lambda x: x not in i)] #temp_df2 = df-temp_df1
        class_length1 = len(temp_df1) 
        class_length2 = len(temp_df2)
        gini = (class_length1/length)*get_gini(temp_df1, label) + (class_length2/length)*get_gini(temp_df2, label) 
        #join을 이용해 변수명 수정해주기
        if len(i) == 1:
            result[i[0]] = gini
        else:
            new_key = '_'.join(i)
            result[new_key] = gini        
    return result

In [42]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged_senior': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'youth_senior': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [43]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [44]:
##문제1 답안
get_binary_split(pd_data,"income")

[['high'],
 ['medium', 'low'],
 ['medium'],
 ['high', 'low'],
 ['low'],
 ['high', 'medium']]

In [51]:
##문제2 답안
for col in pd_data.columns[:-1]:
    print(col,min(get_attribute_gini_index(pd_data, col, "class_buys_computer").items()))

age ('middle_aged', 0.35714285714285715)
income ('high', 0.4428571428571429)
student ('no', 0.3673469387755103)
credit_rating ('excellent', 0.42857142857142855)


 - age가 가장 중요한 변수, age ('middle_aged', 0.35714285714285715)

In [60]:
##문제3 답안
split_df1 = pd_data[pd_data['age'].map(lambda x: x in ['middle_aged'])] 
split_df2 = pd_data[pd_data['age'].map(lambda x: x not in ['middle_aged'])]
new_cols = split_df1.columns[1:-1]
print('middle_aged case')
for col in new_cols:
    print(col,min(get_attribute_gini_index(split_df1, col, "class_buys_computer").items()))
print('---------------------------')
print('Not middle_aged case')
for col in new_cols:
    print(col,min(get_attribute_gini_index(split_df2, col, "class_buys_computer").items()))

middle_aged case
income ('high', 0.0)
student ('no', 0.0)
credit_rating ('excellent', 0.0)
---------------------------
Not middle_aged case
income ('high', 0.375)
student ('no', 0.31999999999999984)
credit_rating ('excellent', 0.4166666666666667)


 - middle_aged dataframe은 label이 모두 yes이므로 gini_index 값이 모두 0이 나와서 더 split할 필요가 없다
 - Not middle_age_dataframe에서는 'student'가 가장 중요한 변수이고, student ('no', 0.31999999999999984)이다.